In [435]:
import heapq
import os
import pickle
import math
import unittest
import random
import matplotlib.pyplot as plt
import itertools

from networkx import Graph
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go

output_notebook()

Loading BokehJS ...

In [2]:
ls

Uninformed & Informed Search Algorithms.ipynb
atlanta_osm.pickle
bfs.py
romania_graph.pickle*


# Priority Queue Implementation

Before diving into implementing our uninformed and informed search algorithms, we need to implement a priority queue to handle the implementation details. For simplicity, we can use Python Lists and Heap objects in order to handle the internals

In [3]:
class PriorityQueue(object):
    """
    A queue structure where each element is served in order of priority.

    Elements in the queue are popped based on the priority with higher priority
    elements being served before lower priority elements.  If two elements have
    the same priority, they will be served in the order they were added to the
    queue.

    Traditionally priority queues are implemented with heaps, but there are any
    number of implementation options.

    (Hint: take a look at the module heapq)

    Attributes:
        queue (list): Nodes added to the priority queue.
    """

    def __init__(self):
        """Initialize a new Priority Queue."""

        self.queue = []
        self.entry_lookup = {}
        self.entry_count = 0

        # placeholder for removed tasks
        self.removed = "REMOVED"

    def pop(self):
        """
        Pop top priority node from queue.

        Returns:
            The node with the highest priority.
        """
        # Check if queue is empty. If not, pop the first item in the PriorityQueue
        if self.size() > 0:
            node = heapq.heappop(self.queue)
            
            if len(node) == 3:            
                # Delete the reference to the node from the entry_lookup dictionary/table
                if node[1] in self.entry_lookup:
                    del self.entry_lookup[node[1]]
                    return (node[0], node[-1])

            elif len(node) == 4:
                if node[2] in self.entry_lookup:
                    del self.entry_lookup[node[2]]
                    return (node[0], node[1], node[-1])
            else:
                raise NotImplementedError

        return None

    def remove(self, node):
        """
        Remove a node from the queue.

        Hint: You might require this in ucs. However, you may
        choose not to use it or to define your own method.

        Args:
            node (tuple): The node to remove from the queue.
        """
#         # Handle Node properties
#         if len(node) > 3:
#             raise NotImplementedError
        
        # Removed the node from the queue and mark it as removed 
        #from the entry_lookup table
        # Referenced from: https://docs.python.org/3/library/heapq.html#priority-queue-implementation-notes
        self.queue.remove(node)

        # Remove the reference to the node
        entry_val = node[1]
        self.entry_lookup[entry_val] = self.removed
        
        # After changing the order of the heap, need to call heapify again
        # https://stackoverflow.com/questions/36876645/unusual-result-from-heappop
        heapq.heapify(self.queue)

    def __iter__(self):
        """Queue iterator."""

        return iter(sorted(self.queue))

    def __str__(self):
        """Priority Queue to string."""

        return "PQ:%s" % self.queue

    def append(self, node):
        """
        Append a node to the queue.

        Args:
            node: Comparable Object to be added to the priority queue.
        """
        # Only handle nodes of length 2 --> (priority, node value)
        if len(node) == 2:
            # Adding entry count to tuple/node for sorting stability
            # when priorities are the same
            # Referenced here: https://docs.python.org/3/library/heapq.html#priority-queue-implementation-notes
            entry = (node[0], self.entry_count, node[1])
            
        elif len(node) == 3:
            entry = (node[0], node[1], self.entry_count, node[2])
        
        elif len(node) > 3:
            raise NotImplementedError
            
        # Store the Lookup ID for the Node as the Nth entry in the PriorityQueue due to possible
        # duplicate node values being entered
        self.entry_lookup[self.entry_count] = node[1]
        self.entry_count += 1

        heapq.heappush(self.queue, entry)
        return self.queue

    def __contains__(self, key):
        """
        Containment Check operator for 'in'

        Args:
            key: The key to check for in the queue.

        Returns:
            True if key is found in queue, False otherwise.
        """

        return key in [n[-1] for n in self.queue]

    def __eq__(self, other):
        """
        Compare this Priority Queue with another Priority Queue.

        Args:
            other (PriorityQueue): Priority Queue to compare against.

        Returns:
            True if the two priority queues are equivalent.
        """

        return self.queue == other.queue

    def size(self):
        """
        Get the current size of the queue.

        Returns:
            Integer of number of items in queue.
        """

        return len(self.queue)

    def clear(self):
        """Reset queue to empty (no nodes)."""

        self.queue = []

    def top(self):
        """
        Get the top item in the queue.

        Returns:
            The first item stored in the queue.
        """

        return self.queue[0]
            

###################################### Test Priority Queue Implementation ###########################################
class TestPriorityQueue(unittest.TestCase):
    """Test Priority Queue implementation"""

    def test_append_and_pop(self):
        """Test the append and pop functions"""
        queue = PriorityQueue()
        temp_list = []

        for _ in range(10):
            a = random.randint(0, 10000)
            queue.append((a, 'a'))
            temp_list.append(a)

        temp_list = sorted(temp_list)

        for item in temp_list:
            popped = queue.pop()
            self.assertEqual(popped[0], item)

    def test_fifo_property(self):
        "Test the fifo property for nodes with same priority"
        queue = PriorityQueue()
        temp_list = [(1,'b'), (1, 'c'), (1, 'a')]

        for node in temp_list:
            queue.append(node)
        
        for expected_node in temp_list:
            actual_node = queue.pop()
            self.assertEqual(actual_node[-1], expected_node[-1])
            
    
    def test_duplicate_entries(self):
        queue = PriorityQueue()
        temp_list = [(1,'b'), (1, 'c'), (1, 'a'), (2, 'd'), (3, 'b')]
        
        for node in temp_list:
            queue.append(node)
        
        dupe_list = [(1,'b'), (1, 'c')]
        for node in dupe_list:
            queue.append(node)
            
        expected_queue = [
            (1, 0, 'b'), 
            (1, 1, 'c'), 
            (1, 2, 'a'), 
            (2, 3, 'd'), 
            (3, 4, 'b'), 
            (1, 5, 'b'), 
            (1, 6, 'c')
        ]
        
        self.assertEqual(queue.queue, expected_queue)
        

    def test_remove(self):
        queue = PriorityQueue()
        #              0          1         2         3         4
        temp_list = [(1, 'b'), (1, 'c'), (1, 'a'), (2, 'd'), (3, 'b')]
        
        for node in temp_list:
            queue.append(node)
        
        dupe_list = [(1,'b'), (1, 'c')]
        for node in dupe_list:
            queue.append(node)
            
        to_remove_list = [(2, 3, 'd'), (1, 1, 'c')]
        for node in to_remove_list:
            queue.remove(node)
            
            
TestPriorityQueue().test_append_and_pop()
TestPriorityQueue().test_fifo_property()
TestPriorityQueue().test_duplicate_entries()
TestPriorityQueue().test_remove()

In [417]:
class ExplorableGraph(object):
    """
    Keeps track of "explored nodes" i.e. nodes that have been queried from the
    graph.

    Delegates graph operations to a networkx.Graph
    """

    def __init__(self, graph):
        """
        :type graph: Graph
        """
        self.__graph = graph
#         self.__graph = nx.relabel_nodes(self.__graph, new_labels)
        self._explored_nodes = dict([(node, 0) for node in self.__graph.nodes()])

    def explored_nodes(self):
        return self._explored_nodes

    def __getattr__(self, item):
        return getattr(self.__graph, item)

    def reset_search(self):
        self._explored_nodes = dict([(node, 0) for node in self.__graph.nodes()])

    def __iter__(self):
        return self.__graph.__iter__()

    def __getitem__(self, n):
        #self._explored_nodes |= {n}
        if n in self.__graph.nodes():
            self._explored_nodes[n] += 1
        return self.__graph.__getitem__(n)

    def nodes_iter(self, data=False):
        self._explored_nodes = set(self.__graph.nodes_iter())
        return self.__graph.nodes_iter(data)

    def neighbors(self, n):
        if n in self.__graph.nodes():
            self._explored_nodes[n] += 1
        return self.__graph.neighbors(n)
    
    def get_edge_weight(self, u, v):
        return self.__graph.get_edge_data(u, v)['weight']
    
#     def _relabel_nodes(self, new_labels):
#         graph = nx.relabel_nodes(self.__graph, new_labels)
#         self.__graph = graph


In [418]:
romania_cities = dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)
)

romania_map_locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

city_mapping = {
    "a": "Arad",
    "b": "Bucharest",
    "c": "Craiova",
    "d": "Drobeta",
    "e": "Eforie",
    "f": "Fagaras",
    "g": "Giurgiu",
    "h": "Hirsova",
    "i": "Iasi",
    "l": "Lugoj",
    "m": "Mehadia",
    "n": "Neamt",
    "o": "Oradea",
    "p": "Pitesti",
    "r": "Rimnicu",
    "s": "Sibiu",
    "t": "Timisoara",
    "u": "Urziceni",
    "v": "Vaslui",
    "z": "Zerind",
}

for k, d in romania_cities.items():
#     print(k, d)
    for ik in d:
        d[ik] = {'weight': d[ik]}

# nx.Graph(romania_cities)

# Graph Setup using Networkx

In [443]:
with (open("romania_graph.pickle", "rb")) as romania_file:
    romania = pickle.load(romania_file)

graph = ExplorableGraph(romania)
# graph._relabel_nodes()
graph.number_of_nodes()
graph.nodes()

# graph.get_edge_weight("Arad", "Zerind")

# G = nx.Graph(romania_cities)
# nx.set_node_attributes(G, romania_map_locations, 'pos')

# for (u, v, wt) in G.edges.data('weight'):
#     print(u,v,wt)

NodeView(('a', 'z', 's', 't', 'b', 'u', 'p', 'g', 'f', 'c', 'd', 'r', 'm', 'e', 'h', 'i', 'v', 'n', 'l', 'o'))

In [442]:
def create_network_map(graph, start, goal, path, heuristic=None):
    """ Create Plotly network graph highlighting the chosen route from Start to Goal
    
    Params:
        graph:
        start:
        goal:
        search_alg:
        heuristic:
    """
    graph_colors = px.colors.qualitative.Plotly

    START_COLOUR = graph_colors[4]
    GOAL_COLOUR = graph_colors[1]
    PATH_COLOUR = graph_colors[2]
    DEFAULT_COLOUR = graph_colors[0]

    # Capture edge relationships
    edge_x = []
    edge_y = []
    edge_text = []
    weights = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    #Create edge trace with defined properties
        
    edge_trace = go.Scatter(
        x = edge_x, 
        y = edge_y,
        line = dict(
            width=0.5, 
            color='#888'
        ),
        hoverinfo='text',
        mode='lines',
        text=[]
    )

    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)


    node_trace = go.Scatter(
        x = node_x, 
        y = node_y,
        text = [],
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            showscale=False,
            color = ["skyblue"] * len(node_x),
            size = 10,
            line_width = 2
        )
    )

    # NOW : creating the annotations, that will display the node name on the figure
    annotations = []
    edge_annotations = []

    # hover text and color group
    for node in G.nodes:
        x_pos, y_pos = G.nodes[node]["pos"]

        node_info = f"Distance to Goal: 500"

        node_trace['text'] += tuple([node_info])

        # THERE : Annotations is a list of dictionaries with every needed parameter for each node annotation
        annotations.append(
            dict(
                x=x_pos + 8,
                y=y_pos + 5,
                text=node, # node name that will be displayed
                xanchor='right',
                yanchor="bottom",
                xshift=10,
                font=dict(color='black', size=10),
                showarrow=False, arrowhead=1, ax=-10, ay=-10)
            )
        
    for edge in G.edges:
        weight = G.get_edge_data(edge[0], edge[1])["weight"]
        edge_trace['text'] += tuple([weight])
        
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        
        annotations.append(
            dict(
                x=(x1 + x0) / 2,
                y=(y1 + y0) / 2,
                text=weight, # node name that will be displayed
                xanchor='right',
                yanchor="bottom",
                xshift=30,
                font=dict(color='black', size=10),
                showarrow=False, arrowhead=1, ax=-10, ay=-10)
            )
    # Color Node Points in plotly
    colors = []
    for node in G.nodes():
        color = DEFAULT_COLOUR  #default
        if path and node in path:
            color = PATH_COLOUR
        if node == start:
            color = START_COLOUR
        elif node == goal:
            color = GOAL_COLOUR
        colors.append(color)
        
    node_trace['marker']['color'] = colors

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    autosize=True,
                    title='<br>Romania Pathfinding Problem',
                    annotations=annotations,
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=50),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    )
    fig.show()

In [422]:
    # Helper function to track path progress
    ############################################################################
    def backtrace(parent, start, end):
        """ Given a dictionary of parent nodes, backtrace from the end node to the start node

            Parameters:
                parent (dict): A dictionary of parent nodes
                start (str): The starting node
                end (str): The ending node
        """
        # Check if node format (Priority, Entry Count, Node Val)
        # If not, then node is in letter format I.e. "b"
        if len(end) > 1:
            end = end[-1]

        path = [end]

        # While we backtrack through the graph, append the parent node to the path
        while path[-1] != start:
            parent_node = parent[path[-1]]
            path.append(parent_node[-1])

        path.reverse()

        return path

    ############################################################################

In [454]:
def breadth_first_search(graph, start, goal):
    """
    Warm-up exercise: Implement breadth-first-search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start and goal nodes (including both).
    """

    if start == goal:
        return []

    depth = 0  # Use depth as the entry value for prioritizing nodes
    visited = []
    parent = {}
    queue = PriorityQueue()
    queue.append((depth, start))

    while queue.size() > 0:
        node = queue.pop()

        # Goal check
        if node[-1] == goal:
            return backtrace(parent, start, goal)

        # Sort in alphabetical order so the nodes are searched deterministically
        neighbours = sorted(list(graph.neighbors(node[-1])))

        depth += 1
        for n in neighbours:
            if n not in visited:
                parent[n] = node

                if n == goal:
                    return backtrace(parent, start, n)

                # If goal hasn't been reached, keep expanding the frontier
                queue.append((depth, n))
                visited.append(n)
                
path = breadth_first_search(graph, start="a", goal="u")
path = [*map(city_mapping.get, path)]

create_network_map(G, 
                   start="Arad", 
                   goal="Urziceni", 
                   path = path
)

In [453]:
def uniform_cost_search(graph, start, goal):
    """
    Warm-up exercise: Implement uniform_cost_search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start and goal nodes (including both).
    """
    #TODO: forgot to add start to visited
    #########
    
    # reset explored nodes on the graph object each time we initiate a search
    graph.reset_search() # for debugging purposes
#     print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
    
    if start == goal:
        return []
    
    distance = 0 # Use depth as the entry value for prioritizing nodes
    visited = []
    parent = {}
    queue = PriorityQueue()
    queue.append((distance, start))
    
    
    while queue.size() > 0:
        node = queue.pop()
        current_dist = node[0]
#         print(f"Processing Node: {node}")
#         print("-------")
        
        # Goal check
        if node[-1] == goal:
#             print("Finding route")
#             print(f"Total cost to goal: {node[0]}")
#             print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
            return backtrace(parent, start, goal)
    
        # Check to prevent cycles. Node a is being checked after a --> z --> a
        # TODO: Check this logic
        if queue.size() > 0 and node[-1] == start:
            continue

        neighbours = sorted(list(graph.neighbors(node[-1])))
#         print(f"Neighbours {neighbours}")
        for neighbour in neighbours:
            path_dist = graph.get_edge_weight(node[-1], neighbour)
#             print(f"Nodes explored: {sum(list(graph.explored_nodes().values()))}")
#             print()
#             print(f"Neighbour: {neighbour} with path distance {path_dist}")
            
            new_dist = current_dist + path_dist
            
            if neighbour not in visited and not any(elem in neighbours for elem in queue):
                parent[neighbour] = node
                visited.append(neighbour)
                queue.append((new_dist, neighbour))

            # Append node if it is already in the queue but has a lower cost than a previously explored node
            else:
                for frontier_node in queue:
                    if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
#                         print(f"Frontier node: {frontier_node}")
#                         print(f"Queue: {queue}")
                        parent[neighbour] = node
                        queue.remove(frontier_node)
                        queue.append((new_dist, neighbour))

path = uniform_cost_search(graph, start="a", goal="b")
path = [*map(city_mapping.get, path)]

create_network_map(G, 
                   start="Arad", 
                   goal="Bucharest", 
                   path = path
)

# A* Search With Different Heuristics

In [455]:
def null_heuristic(graph, v, goal):
    """
    Null heuristic used as a base line. Basically becomes Dijkstra's algorithm 
    instead of A* search

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        v (str): Key for the node to calculate from.
        goal (str): Key for the end node to calculate to.

    Returns:
        0
    """

    return 0


def euclidean_dist_heuristic(graph, v, goal):
    """
    Warm-up exercise: Implement the euclidean distance heuristic.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        v (str): Key for the node to calculate from.
        goal (str): Key for the end node to calculate to.

    Returns:
        Euclidean distance between `v` node and `goal` node
    """
    start_x, start_y = graph.nodes[v]["pos"]
    goal_x, goal_y = graph.nodes[goal]["pos"]
    
    # Calculate heuristic using euclidean distance since we can move in any direction
    # Code referenced from here: http://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html
    # TODO: not quite sure  about this
    dx = abs(start_x - goal_x) ** 2
    dy = abs(start_y - goal_y) ** 2
    return math.sqrt(dx + dy)

    # end of referenced code
    
print(euclidean_dist_heuristic(graph, "a", "u"))

391.6490776192381


In [457]:
def a_star(graph, start, goal, heuristic=euclidean_dist_heuristic):
    """
    Warm-up exercise: Implement A* algorithm.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start and goal nodes (including both).
    """
    #TODO: forgot to add start to visited
    #########
    
    # reset explored nodes on the graph object each time we initiate a search
    graph.reset_search() # for debugging purposes
#     print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
    
    if start == goal:
        return []

    init_path_dist = 0
    visited = []
    parent = {}
    queue = PriorityQueue()
    queue.append((heuristic(graph, start, goal), init_path_dist, start))
    
    while queue.size() > 0:
#         print(f"Queue: {queue}")
        node = queue.pop()
#         print(f"Node")
        current_dist = node[1]
#         print()
#         print(f"Processing Node: {node}")
#         print("-------")
        
       # Goal check
        if node[-1] == goal:
#             print("Finding route")
#             print(f"Total cost to goal: {node[0]}")
#             print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
            return backtrace(parent, start, goal)
    
        neighbours = sorted(list(graph.neighbors(node[-1])))
#         print(f"Neighbours {neighbours}")
        for neighbour in neighbours:
            # g-value
            # TODO: something wrong happening here
            path_dist = graph.get_edge_weight(node[-1], neighbour)
            dist = current_dist + path_dist
            h = heuristic(graph, neighbour, goal)
            f =  dist + h
#             print(f"F Value from {node} --> {neighbour}: {current_dist} + {path_dist} + {h} == {f}")
            
            if neighbour not in visited and not any(elem in neighbours for elem in queue):
                parent[neighbour] = node
                visited.append(neighbour)
                queue.append((f, dist, neighbour))
                
            # Append node if it is already in the queue but has a lower cost than a previously explored node
            else:
                for frontier_node in queue:
                    if frontier_node[-1] == neighbour and f < frontier_node[0]:
                        parent[neighbour] = node
                        queue.remove(frontier_node)
                        queue.append((f, dist, neighbour))
                
            
#             print(f"F-value for Neighbour {neighbour} == {f}")
#         print(f"Queue: {queue}")
#             print(f"Nodes explored: {sum(list(graph.explored_nodes().values()))}")
#             print()

path = a_star(graph, start="a", goal="u")

path = [*map(city_mapping.get, path)]

create_network_map(G, 
                   start="Arad", 
                   goal="Urziceni",
                   path = path
)

# Bi-directional Search

In [459]:
# Helper function to track path progress
############################################################################
def bidirectional_backtrace(start_parent, goal_parent, intersecting_node, start, goal):
    """ Given a dictionary of parent nodes, backtrace from the end node to the start node

        Parameters:
            start_parent (dict): A dictionary of parent nodes for search originating from the start node
            goal_parent (dict): A dictionary of parent nodes for search originating from the goal node
            intersection (string): Node where the two searches intersect
            start (str): The starting node
            end (str): The ending node
    """ 
    solution = []
    total_cost = 0
    
    
    # Backtrace Forward search
    for_path = [intersecting_node]
    
    while for_path[-1] != start:
        parent_node = start_parent[for_path[-1]]
        parent_node, path_cost = parent_node[0], parent_node[1]
        for_path.append(parent_node[-1])

    for_path.reverse()
    # Remove extraneous intersecting node from the path since it's included in the reverse search
    solution.extend(for_path[:-1])

    # Backtrace reverse search
    rev_path = [intersecting_node]
#     print(f"Formulating path for goal search. Starting from {rev_path}")

    # While we backtrack through the graph, append the parent node to the path
    while rev_path[-1] != goal:
        parent_node = goal_parent[rev_path[-1]]
        parent_node, path_cost = parent_node[0], parent_node[1]
        rev_path.append(parent_node[-1])

    solution.extend(rev_path)
    return solution

############################################################################

def nodes_in_queue(queue):
    values = [node[-1] for node in queue]
    return values

def bidirectional_ucs(graph, start, goal):
    """
    Exercise 1: Bidirectional Search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start and goal nodes (including both).
    """
    #TODO: Check termination conditions. May have to change meet in the middle conditions to visited set (explored)
    # and not the frontier
    
    # reset explored nodes on the graph object each time we initiate a search
    graph.reset_search() # for debugging purposes
#     print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
#     print()
    
    if start == goal:
        return []
    
    
    forward_dist = 0 # Use depth as the entry value for prioritizing nodes --> start --> goal
    reverse_dist = 0 # Use depth as the entry value for prioritizing nodes --> goal --> start
    
    forward_visited = []
    reverse_visited = []
    
    forward_parent = {}
    reverse_parent = {}
    
    forward_queue = PriorityQueue()
    reverse_queue = PriorityQueue()
    
    forward_queue.append((forward_dist, start))
    forward_visited.append(start)
    
    reverse_queue.append((reverse_dist, goal))
    reverse_visited.append(goal)
    
    total_path = []
    
    while forward_queue.size() > 0 and reverse_queue.size() > 0:
        
        if forward_queue.size() > 0:
            node = forward_queue.pop()
            forward_dist = node[0]
#             print(f"Forward: Processing Node {node}")
#             print("--------------------------------")

            # Goal check
            if node[-1] == goal or node[-1] in reverse_queue:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                reverse_queue_nodes = nodes_in_queue(reverse_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                reverse_union = set().union(reverse_queue_nodes, reverse_visited)
                
                # Find insersection between forward_union and explored set of current search
                frontier_intersection = list(reverse_union.intersection(set(forward_visited)))
                frontier_intersection.append(node[-1])
#                 print(f"Forward Search issue")
                
                return bidirectional_backtrace(forward_parent, reverse_parent, node[-1], start, goal)
            
            neighbours = sorted(list(graph.neighbors(node[-1])))
#             print(f"Neighbours {neighbours}")
            for neighbour in neighbours:
                path_dist = graph.get_edge_weight(node[-1], neighbour)
#                 print(f"Path from {node[-1]} to {neighbour} with path cost {path_dist} + forward distance {forward_dist}")
                
                # Compute total distance so far
                new_dist = forward_dist + path_dist
                if neighbour not in forward_visited: #and not any(elem in neighbours for elem in queue):
                    forward_parent[neighbour] = (node, path_dist)
                    forward_visited.append(neighbour)
                    forward_queue.append((new_dist, neighbour))
                    
                # Append node if it is already in the queue but has a lower cost than a previously explored node
                else:
                    for frontier_node in forward_queue:
                        if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
    #                         print(f"Frontier node: {frontier_node}")
                            forward_parent[neighbour] = (node, path_dist) #node
                            forward_queue.remove(frontier_node)
                            forward_queue.append((new_dist, neighbour))

#             print(f"Forward Queue: {forward_queue}")    
#             print()
            
        if reverse_queue.size() > 0:
            node = reverse_queue.pop()
            reverse_dist = node[0]
#             print(f"Reverse: Processing Node {node}")
#             print("--------------------------------")

            # Goal check
            if node[-1] == start or node[-1] in forward_queue:
#                 print(f"Goal found or found in opposing search PQ. Node {node[-1]}")
                forward_queue_nodes = nodes_in_queue(forward_queue)
#                 print(f"Nodes in forward frontier: {forward_queue_nodes}")

                # Union of opposite search frontier and explored sets
                forward_union = set().union(forward_queue_nodes, forward_visited)
#                 print(f"Forward Union: {forward_union}")
                
                # Find insersection between forward_union and explored set of current search
                frontier_intersection = list(forward_union.intersection(set(reverse_visited)))
                frontier_intersection.append(node[-1])
#                 print(f"Frontier intersection: {frontier_intersection}")
                
                return bidirectional_backtrace(forward_parent, reverse_parent, node[-1], start, goal)
                
#                 rev_path = backtrace(reverse_parent, goal, node[-1])
#                 print(f"Reverse path {rev_path}")
#                 print(f"Total Path: {total_path}")
#                 return forward_parent, reverse_parent, goal, node[-1]
            
            neighbours = sorted(list(graph.neighbors(node[-1])))
#             print(f"Neighbours {neighbours}")
            for neighbour in neighbours:
                path_dist = graph.get_edge_weight(node[-1], neighbour)
#                 print(f"Path from {node[-1]} to {neighbour} with path cost {path_dist}")
                
                # Compute total distance so far
                new_dist = reverse_dist + path_dist
                if neighbour not in reverse_visited: #and not any(elem in neighbours for elem in queue):
                    reverse_parent[neighbour] = (node, path_dist) #node
                    reverse_visited.append(neighbour)
                    reverse_queue.append((new_dist, neighbour))
                    
                # Append node if it is already in the queue but has a lower cost than a previously explored node
                else:
                    for frontier_node in forward_queue:
                        if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
    #                         print(f"Frontier node: {frontier_node}")
                            reverse_parent[neighbour] = (node, path_dist) #node
                            reverse_queue.remove(frontier_node)
                            reverse_queue.append((new_dist, neighbour))
                
#             print(f"Reverse Queue: {reverse_queue}")
#             print()
                    
            
            
        
path = bidirectional_ucs(graph, start="a", goal="u")
path = [*map(city_mapping.get, path)]

create_network_map(G, 
                   start="Arad", 
                   goal="Urziceni", 
                   path = path
)
# print()
# print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")

# forward_parent,reverse_parent, goal, intersecting_node = bidirectional_ucs(graph, start="a", goal="u")

# print(bidirectional_ucs(graph, start="c", goal="r"))

In [463]:
def bidirectional_a_star(graph, start, goal, heuristic=euclidean_dist_heuristic):
    """
    Exercise 2: Bidirectional A*.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start and goal nodes (including both).
    """
    #TODO: Check termination conditions. May have to change meet in the middle conditions to visited set (explored)
    # and not the frontier

    # reset explored nodes on the graph object each time we initiate a search
    graph.reset_search() # for debugging purposes
    print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
    
    if start == goal:
        return []

    forward_init_path_dist = 0
    forward_visited = []
    forward_parent = {}
    forward_queue = PriorityQueue()
    forward_queue.append((heuristic(graph, start, goal), forward_init_path_dist, start))
    
    reverse_init_path_dist = 0
    reverse_visited = []
    reverse_parent = {}
    reverse_queue = PriorityQueue()
    reverse_queue.append((heuristic(graph, goal, start), reverse_init_path_dist, goal))
    
    while forward_queue.size() > 0 and reverse_queue.size() > 0:
        
        if forward_queue.size() > 0:
            node = forward_queue.pop()
            forward_dist = node[0]
#             print(f"Forward: Processing Node {node}")
#             print("--------------------------------")

            # Goal check
            if node[-1] == goal or node[-1] in reverse_queue:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                reverse_queue_nodes = nodes_in_queue(reverse_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                reverse_union = set().union(reverse_queue_nodes, reverse_visited)
                
                # Find insersection between forward_union and explored set of current search
                frontier_intersection = list(reverse_union.intersection(set(forward_visited)))
                frontier_intersection.append(node[-1])
                
                return bidirectional_backtrace(forward_parent, reverse_parent, node[-1], start, goal)

            neighbours = sorted(list(graph.neighbors(node[-1])))
            print(f"Neighbours {neighbours}")
            for neighbour in neighbours:
                # g-value
                path_dist = graph.get_edge_weight(node[-1], neighbour)
                dist = forward_dist + path_dist
                h = heuristic(graph, neighbour, goal)
                f =  dist + h
                print(f"F Value from {node} --> {neighbour}: {forward_dist} + {path_dist} + {h} == {f}")

                if neighbour not in forward_visited and not any(elem in neighbours for elem in forward_queue):
                    forward_parent[neighbour] = node
                    forward_visited.append(neighbour)
                    forward_queue.append((f, dist, neighbour))

                # Append node if it is already in the queue but has a lower cost than a previously explored node
                else:
                    for frontier_node in forward_queue:
                        if frontier_node[-1] == neighbour and f < frontier_node[0]:
                            forward_parent[neighbour] = node
                            forward_queue.remove(frontier_node)
                            forward_queue.append((f, dist, neighbour))
                            
        if reverse_queue.size() > 0:
            node = reverse_queue.pop()
            reverse_dist = node[0]
            print(f"Reverse: Processing Node {node}")
            print("--------------------------------")

            # Goal check
            if node[-1] == start or node[-1] in forward_queue:
                print(f"Goal found or found in opposing search PQ. Node {node[-1]}")
                forward_queue_nodes = nodes_in_queue(forward_queue)
                print(f"Nodes in forward frontier: {forward_queue_nodes}")

                # Union of opposite search frontier and explored sets
                forward_union = set().union(forward_queue_nodes, forward_visited)
#                 print(f"Forward Union: {forward_union}")
                
                # Find insersection between forward_union and explored set of current search
                frontier_intersection = list(forward_union.intersection(set(reverse_visited)))
                frontier_intersection.append(node[-1])
#                 print(f"Frontier intersection: {frontier_intersection}")
                
                return bidirectional_backtrace(forward_parent, reverse_parent, node[-1], start, goal)
    
                            
            neighbours = sorted(list(graph.neighbors(node[-1])))
            print(f"Neighbours {neighbours}")
            for neighbour in neighbours:
                # g-value
                # TODO: something wrong happening here
                path_dist = graph.get_edge_weight(node[-1], neighbour)
                dist = reverse_dist + path_dist
                h = heuristic(graph, neighbour, goal)
                f =  dist + h
                print(f"F Value from {node} --> {neighbour}: {reverse_dist} + {path_dist} + {h} == {f}")

                if neighbour not in reverse_visited and not any(elem in neighbours for elem in reverse_queue):
                    reverse_parent[neighbour] = node
                    reverse_visited.append(neighbour)
                    reverse_queue.append((f, dist, neighbour))

                # Append node if it is already in the queue but has a lower cost than a previously explored node
                else:
                    for frontier_node in reverse_queue:
                        if frontier_node[-1] == neighbour and f < frontier_node[0]:
                            reverse_parent[neighbour] = node
                            reverse_queue.remove(frontier_node)
                            reverse_queue.append((f, dist, neighbour))
                            
                            
print(bidirectional_a_star(graph, start="a", goal="u", heuristic=euclidean_dist_heuristic))


Total Nodes Explored: 0
Neighbours ['s', 't', 'z']
F Value from (391.6490776192381, 0, 'a') --> s: 391.6490776192381 + 140 + 271.01660465735307 == 802.6656822765913
F Value from (391.6490776192381, 0, 'a') --> t: 391.6490776192381 + 118 + 366.9386869764484 == 876.5877645956865
F Value from (391.6490776192381, 0, 'a') --> z: 391.6490776192381 + 75 + 392.2562937672256 == 858.9053713864637
Reverse: Processing Node (391.6490776192381, 0, 'u')
--------------------------------
Neighbours ['b', 'h', 'v']
F Value from (391.6490776192381, 0, 'u') --> b: 391.6490776192381 + 85 + 60.53924347066124 == 537.1883210898993
F Value from (391.6490776192381, 0, 'u') --> h: 391.6490776192381 + 98 + 78.0 == 567.6490776192381
F Value from (391.6490776192381, 0, 'u') --> v: 391.6490776192381 + 142 + 107.91200118615167 == 641.5610788053898
Neighbours ['a', 'f', 'o', 'r']
F Value from (802.6656822765913, 531.6490776192381, 's') --> a: 802.6656822765913 + 140 + 391.6490776192381 == 1334.3147598958294
F Value fr

# Tridirectional Search

In [462]:
# Helper function to track path progress
############################################################################
def bidirectional_backtrace(start_parent, goal_parent, intersecting_node, start, goal):
    """ Given a dictionary of parent nodes, backtrace from the end node to the start node

        Parameters:
            start_parent (dict): A dictionary of parent nodes for search originating from the start node
            goal_parent (dict): A dictionary of parent nodes for search originating from the goal node
            intersection (string): Node where the two searches intersect
            start (str): The starting node
            end (str): The ending node
    """ 
    solution = []
    forward_cost = 0
    reverse_cost = 0
    
    
    # Backtrace Forward search
    for_path = [intersecting_node]
    
    while for_path[-1] != start:
        parent_node = start_parent[for_path[-1]]
#         print(f"Parent Node: {parent_node}, start node: {for_path[-1]}")
        for_path.append(parent_node[-1])

    for_path.reverse()
    # Remove extraneous intersecting node from the path since it's included in the reverse search
    solution.extend(for_path[:-1])

    # Backtrace reverse search
    rev_path = [intersecting_node]
#     print(f"Formulating path for goal search. Starting from {rev_path}")

    # While we backtrack through the graph, append the parent node to the path
    while rev_path[-1] != goal:
        parent_node = goal_parent[rev_path[-1]]
        rev_path.append(parent_node[-1])

    solution.extend(rev_path)
    return solution

def nodes_in_queue(queue):
    values = [node[-1] for node in queue]
    return values

def tri_a_star_final(goals, solution_12, solution_23, solution_31):
    solutions = [solution_12, solution_23, solution_31]
    
    cost_min = float("inf")
    combo = None
    for sol in solutions:
        if set(goals).issubset(sol[0]) and sol[1] < cost_min:
            cost_min = sol[1]
            solution = sol[0]
    
    # Find minimum cost combination
    print('12 + 23: ', solution_12[1] + solution_23[1])
    print('23 + 31: ', solution_23[1] + solution_31[1])
    print('31 + 12: ', solution_31[1] + solution_12[1])
    if solution_12[1] + solution_23[1] < cost_min:
        cost_min = solution_12[1] + solution_23[1]
        combo = (solution_12[0], solution_23[0])
    if solution_23[1] + solution_31[1] < cost_min:
        cost_min = solution_23[1] + solution_31[1]
        combo = (solution_23[0], solution_31[0])
    if solution_31[1] + solution_12[1] < cost_min:
        cost_min = solution_31[1] + solution_12[1]
        combo = (solution_31[0], solution_12[0])

    # Stitch best combo together
    if combo is None:
        return solution
    if combo[0][-1] == combo[1][0]:
        solution = combo[0] + combo[1][1:]
    else:
        solution = combo[0] + combo[1]
    return solution

############################################################################

def tridirectional_search(graph, goals):
    """
    Implement tridirectional search in the naive way: starting from each goal node, perform a 
    uniform-cost search and keep expanding until two of the three searches meet. This should 
    be one continuous path that connects all three nodes
    
    Exercise 3: Tridirectional UCS Search

    See README.MD for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        goals (list): Key values for the 3 goals

    Returns:
        The best path as a list from one of the goal nodes (including both of
        the other goal nodes)
        
    TODO:
        - Can probably use backtrace() instead of bidirectional backtrace
        - Define functions for repeated code
        - simplify logic
    """
    
    # reset explored nodes on the graph object each time we initiate a search
    graph.reset_search() # for debugging purposes
    print(f"Total Nodes Explored: {sum(list(graph.explored_nodes().values()))}")
    
    # Check if all paths are the same
    if len(set(goals)) == 1:
        return []

    
    s1_goal = goals[0]
    s2_goal = goals[1]
    s3_goal = goals[2]

    s1_dist = 0 # Use depth as the entry value for prioritizing nodes --> start --> goal
    s2_dist = 0 # Use depth as the entry value for prioritizing nodes --> goal --> start
    s3_dist = 0
    
    s1_visited = []
    s2_visited = []
    s3_visited = []
    
    s1_parent = {}
    s2_parent = {}
    s3_parent = {}
    
    solution_12 = None
    solution_23 = None
    solution_31 = None
    
    s1_queue = PriorityQueue()
    s2_queue = PriorityQueue()
    s3_queue = PriorityQueue()
    
    s1_queue.append((s1_dist, goals[0]))
    s1_visited.append(goals[0])
    
    s2_queue.append((s2_dist, goals[1]))
    s2_visited.append(goals[1])

    s3_queue.append((s3_dist, goals[2]))
    s3_visited.append(goals[2])
    
    s1_size = s1_queue.size()
    s2_size = s1_queue.size()
    s3_size = s1_queue.size()
    
    # Loop until we found a solution between all three goals is found
    while solution_12 is None or solution_23 is None or solution_31 is None:
        
        #######################################################################################
        # Check for path between Goal 1 and Goal 2
        if min(s1_size, s2_size, s3_size) == s1_size and solution_12 is None:
            print(f"Searching Agent #1")

            node = s1_queue.pop()
            path_dist = node[0]
            print(f"Forward: Processing Node {node}")
            print("--------------------------------")

            # Goal 1 Checl See if intersection occurs between S2 or S3
            if node[-1] == s2_goal or node[-1] in s2_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s2_queue_nodes = nodes_in_queue(s2_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s2_union = set().union(s2_queue_nodes, s2_visited)
                
                # Find insersection between S2 and explored set of current search
                frontier_intersection = list(s2_union.intersection(set(s1_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S1 and S2 Searches Intersect")
                    
                solution_12 = bidirectional_backtrace(s1_parent, s2_parent, node[-1], s1_goal, s2_goal)
                print(f"Solution_12: {solution_12}")
                
                s1_size = float("inf")
                
            elif node[-1] == s3_goal or node[-1] in s3_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s3_queue_nodes = nodes_in_queue(s3_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s3_union = set().union(s3_queue_nodes, s3_visited)
                
                # Find insersection between S2 and explored set of current search
                frontier_intersection = list(s3_union.intersection(set(s1_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S1 and S3 Searches Intersect")
                    
                solution_31 = bidirectional_backtrace(s1_parent, s3_parent, node[-1], s1_goal, s3_goal)
                print(f"Solution_31: {solution_31}")
                s3_size = float("inf")
                
            if s1_size != float("inf"):
                neighbours = sorted(list(graph.neighbors(node[-1])))
    #             print(f"Neighbours {neighbours}")
                for neighbour in neighbours:
                    path_dist = graph.get_edge_weight(node[-1], neighbour)
    #                 print(f"Path from {node[-1]} to {neighbour} with path cost {path_dist}")

                    # Compute total distance so far
                    new_dist = s1_dist + path_dist
                    if neighbour not in s1_visited and not any(elem in neighbours for elem in s1_queue):
                        s1_parent[neighbour] = node #(node, path_dist)
                        s1_visited.append(neighbour)
                        s1_queue.append((new_dist, neighbour))

                    # Append node if it is already in the queue but has a lower cost than a previously explored node
                    else:
                        for frontier_node in s1_queue:
                            if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
        #                         print(f"Frontier node: {frontier_node}")
                                s1_parent[neighbour] = node #(node, path_dist)
                                s1_queue.remove(frontier_node)
                                s1_queue.append((new_dist, neighbour))
                
                if s1_size != float("inf"):
                    s1_size = s1_queue.size()
            
            
        #######################################################################################
        # Check for path between goal 2 and goal 3
        if min(s1_size, s2_size, s3_size) == s2_size and solution_23 is None:
            print(f"Searching Agent #2")
            
            node = s2_queue.pop()
            path_dist = node[0]
            
            print(f"Forward: Processing Node {node}")
            print("--------------------------------")

            # Goal check for Goal 2: See if inttersection occurs between S2 and S3 or S1
            if node[-1] == s1_goal or node[-1] in s1_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s1_queue_nodes = nodes_in_queue(s1_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s1_union = set().union(s1_queue_nodes, s1_visited)
                
                # Find insersection between S2 and explored set of current search
                frontier_intersection = list(s1_union.intersection(set(s2_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S2 and S1 Searches Intersect")
                    
                solution_12 = bidirectional_backtrace(s2_parent, s1_parent, node[-1], s2_goal, s1_goal)
                print(f"Solution_12: {solution_12}")
                s1_size = float("inf")
                
            elif node[-1] == s3_goal or node[-1] in s3_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s3_queue_nodes = nodes_in_queue(s3_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s3_union = set().union(s3_queue_nodes, s3_visited)
                
                # Find insersection between S2 and explored set of current search
                frontier_intersection = list(s3_union.intersection(set(s2_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S2 and S3 Searches Intersect")
                    
                solution_23 = bidirectional_backtrace(s2_parent, s3_parent, node[-1], s2_goal, s3_goal)
                print(f"Solution_23: {solution_23}")
                
                s3_size = float("inf")
                
            if s2_size != float("inf"):
                neighbours = sorted(list(graph.neighbors(node[-1])))
    #             print(f"Neighbours {neighbours}")
                for neighbour in neighbours:
                    path_dist = graph.get_edge_weight(node[-1], neighbour)
    #                 print(f"Path from {node[-1]} to {neighbour} with path cost {path_dist}")

                    # Compute total distance so far
                    new_dist = s2_dist + path_dist
                    if neighbour not in s2_visited and not any(elem in neighbours for elem in s2_queue):
                        s2_parent[neighbour] = node #(node, path_dist)
                        s2_visited.append(neighbour)
                        s2_queue.append((new_dist, neighbour))

                    # Append node if it is already in the queue but has a lower cost than a previously explored node
                    else:
                        for frontier_node in s2_queue:
                            if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
        #                         print(f"Frontier node: {frontier_node}")
                                s2_parent[neighbour] = node #(node, path_dist)
                                s2_queue.remove(frontier_node)
                                s2_queue.append((new_dist, neighbour))
                
                if s2_size != float("inf"):
                    s2_size = s1_queue.size()
    
        #######################################################################################
        # Check for path between goal 1 and goal 3
        if min(s1_size, s2_size, s3_size) == s3_size and solution_31 is None:

            print(f"Searching Agent #3")
            
            node = s3_queue.pop()
            path_dist = node[0]
            print(f"Forward: Processing Node {node}")
            print("--------------------------------")

            # Goal check for S3. See if intersection occurs between S3 and S1 or S2
            if node[-1] == s1_goal or node[-1] in s1_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s1_queue_nodes = nodes_in_queue(s1_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s1_union = set().union(s1_queue_nodes, s1_visited)
                
                # Find insersection between S3 and explored set of current search
                frontier_intersection = list(s1_union.intersection(set(s3_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S3 and S1 Searches Intersect")
                    
                solution_31 = bidirectional_backtrace(s3_parent, s1_parent, node[-1], s3_goal, s1_goal)
                print(f"Solution_31: {solution_31}")
                s1_size = float("inf")
                
            # Goal check to find intersection of S3 and S2
            elif node[-1] == s2_goal or node[-1] in s2_visited:
#                 print(f"Goal found or found in opposing search PQ: {node[-1]}")
                
                s2_queue_nodes = nodes_in_queue(s2_queue)

                # Union of opposite search frontier and explored sets
                #TODO: can probably rewrite this
                s2_union = set().union(s2_queue_nodes, s2_visited)
                
                # Find insersection between S2 and explored set of current search
                frontier_intersection = list(s2_union.intersection(set(s3_visited)))
                
                if node[-1] not in frontier_intersection:
                    frontier_intersection.append(node[-1])
                    
                    print(f"S3 and S2 Searches Intersect")
                    
                solution_23 = bidirectional_backtrace(s3_parent, s2_parent, node[-1], s3_goal, s2_goal)
                print(f"Solution_23: {solution_23}")
                s2_size = float("inf")
                
            if s3_size != float("inf"):
                neighbours = sorted(list(graph.neighbors(node[-1])))
    #             print(f"Neighbours {neighbours}")
                for neighbour in neighbours:
                    path_dist = graph.get_edge_weight(node[-1], neighbour)
    #                 print(f"Path from {node[-1]} to {neighbour} with path cost {path_dist}")

                    # Compute total distance so far
                    new_dist = s3_dist + path_dist
                    if neighbour not in s3_visited and not any(elem in neighbours for elem in s3_queue):
                        s3_parent[neighbour] = node #(node, path_dist)
                        s3_visited.append(neighbour)
                        s3_queue.append((new_dist, neighbour))

                    # Append node if it is already in the queue but has a lower cost than a previously explored node
                    else:
                        for frontier_node in s1_queue:
                            if frontier_node[-1] == neighbour and new_dist < frontier_node[0]:
        #                         print(f"Frontier node: {frontier_node}")
                                s3_parent[neighbour] = node #(node, path_dist)
                                s3_queue.remove(frontier_node)
                                s3_queue.append((new_dist, neighbour))
                
                if s3_size != float("inf"):
                    s3_size = s1_queue.size()
        
    return solution_12, solution_23, solution_31
    
keys = graph.nodes.keys()
triplets = list(itertools.permutations(keys, 3))
goals = triplets[0]

print(f"Finding path between Goals: {goals}")
print(tridirectional_search(graph, goals))



Finding path between Goals: ('a', 'z', 's')
Total Nodes Explored: 0
Searching Agent #1
Forward: Processing Node (0, 'a')
--------------------------------
Searching Agent #2
Forward: Processing Node (0, 'z')
--------------------------------
Solution_12: ['z', 'a']
Searching Agent #3
Forward: Processing Node (0, 's')
--------------------------------
Solution_31: ['s', 'a']
Searching Agent #2
Forward: Processing Node (71, 'o')
--------------------------------
Solution_23: ['z', 'o', 's']
(['z', 'a'], ['z', 'o', 's'], ['s', 'a'])


In [250]:
def bidirectional_backtrace(start_parent, goal_parent, intersecting_node, start, goal):
    """ Given a dictionary of parent nodes, backtrace from the end node to the start node

        Parameters:
            start_parent (dict): A dictionary of parent nodes for search originating from the start node
            goal_parent (dict): A dictionary of parent nodes for search originating from the goal node
            intersection (string): Node where the two searches intersect
            start (str): The starting node
            end (str): The ending node
    """ 
    solution = []
    forward_cost = 0
    reverse_cost = 0
    
    
    # Backtrace Forward search
    for_path = [intersecting_node]
    
    while for_path[-1] != start:
        parent_node = start_parent[for_path[-1]]
        for_path.append(parent_node[-1])

    for_path.reverse()
    # Remove extraneous intersecting node from the path since it's included in the reverse search
    solution.extend(for_path[:-1])

    # Backtrace reverse search
    rev_path = [intersecting_node]
#     print(f"Formulating path for goal search. Starting from {rev_path}")

    # While we backtrack through the graph, append the parent node to the path
    while rev_path[-1] != goal:
        parent_node = goal_parent[rev_path[-1]]
        rev_path.append(parent_node[-1])

    solution.extend(rev_path)
    return solution



# def compute_final_tri_star_path(graph, goals, solution_12, solution_23, solution_31):
#     """ Take multiple paths returned from Tridirectional Search and return the best/cost
#         optimal path
        
#         Parameters:
#             graph: networkx graph
#             goals: Nodes in the graph for which we are finding a path between
#             solution_12: Path between goal 1 and 2
#             solution_23: Path between goal 2 and 3
#             solution_31: Path between goal 3 and 1
#     """
#     solutions = [solution_12, solution_23, solution_31]
    
#     cost_min = float("inf")
#     combo = None
    
#     for sol in solutions:
#         if set(goals).issubset(sol[0]) and sol[1] < cost_min:
#             cost_min = sol[1]
#             solution = sol[0]
    
    
    
#     return

solution_12, solution_23, solution_31 = tridirectional_search(graph, goals)
# print(compute_final_tri_star_path(goals, solution_12, solution_23, solution_31))

Total Nodes Explored: 0
Searching Agent #1
Forward: Processing Node (0, 'a')
--------------------------------
Searching Agent #2
Forward: Processing Node (0, 'z')
--------------------------------
Solution_12: ['z', 'a']
Searching Agent #3
Forward: Processing Node (0, 's')
--------------------------------
Solution_31: ['s', 'a']
Searching Agent #2
Forward: Processing Node (71, 'o')
--------------------------------
Solution_23: ['z', 'o', 's']


# Visualizations for Search Algorithms